In [1]:
import pandas as pd
import json
import warnings

# Отключение предупреждений
warnings.filterwarnings("ignore", category=UserWarning, message="Parsing dates")

# Чтение файла data.csv
data_df = pd.read_csv('data.csv')

In [2]:
data_df

,id,application_date,contracts
0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN
1,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212.0,2024-02-12 19:24:41.493000+00:00,NaN
3,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214.0,2024-02-12 19:24:56.857000+00:00,NaN
...,...,...,...
995,2926205.0,2024-02-13 06:09:54.210000+00:00,"[{""contract_id"": 18410, ""bank"": ""004"", ""summa""..."
996,2926206.0,2024-02-13 06:09:54.306000+00:00,NaN
997,2926207.0,2024-02-13 06:09:55.661000+00:00,NaN
998,2926208.0,2024-02-13 06:09:57.024000+00:00,NaN


данные в contracts представляют собой записи о кредитных заявках и контрактах, поданных в различные банки. Они дают возможность понять, сколько заявок на кредиты подано, сколько из них перешли в стадию заключения контракта, и в каких банках это произошло. Данные включают уникальные идентификаторы заявок и контрактов, коды банков, суммы кредитов, даты подачи заявок и заключения контрактов.

Смысл полей:
contract_id: Уникальный номер контракта. Появляется только тогда, когда заявка одобрена и заключен контракт. Пустое значение в этом поле указывает на то, что контракт пока не заключен.
bank: Код банка, в который подана заявка. Каждому банку соответствует свой числовой код.
summa: Сумма кредита, на которую подана заявка или которая одобрена банком. Пустое значение может означать, что сумма не указана или заявка еще не прошла обработку.
loan_summa: Фактическая сумма выданного кредита, которая часто равна нулю, если кредит еще не выдан.
claim_date: Дата подачи заявки на кредит.
claim_id: Уникальный идентификатор заявки. Он может быть как числовым, так и буквенно-числовым. Это различие может указывать на то, что заявки поступили из разных систем или каналов.
contract_date: Дата заключения контракта. Если это поле пустое, то контракт по заявке не заключен.

In [3]:
def calculate_features(row):
    try:
        contracts = json.loads(row['contracts']) if isinstance(row['contracts'], str) else []
    except (TypeError, json.JSONDecodeError):
        contracts = []
    
    # инициализация значений характеристик
    tot_claim_cnt_l180d = -3
    disb_bank_loan_wo_tbc = -1
    day_sinlastloan = -1
    
    # Преобразуем application_date в формат, не учитывающий часовой пояс
    application_date = pd.to_datetime(row['application_date']).tz_localize(None) if pd.notnull(row['application_date']) else None
    # Считаем tot_claim_cnt_l180d 
    if contracts:
        claims = [
            contract for contract in contracts
            if isinstance(contract, dict) and contract.get('claim_id') is not None and contract.get('claim_date')
        ]
        
        claim_dates = [
            pd.to_datetime(claim['claim_date']).tz_localize(None)
            for claim in claims if pd.notnull(claim['claim_date'])]
        
        claim_count_l180d = sum(
            (application_date - claim_date).days <= 180
            for claim_date in claim_dates)
        
        tot_claim_cnt_l180d = claim_count_l180d if claim_count_l180d > 0 else -3
    
    # Считаем disb_bank_loan_wo_tbc
    valid_loans = [
        contract for contract in contracts
        if isinstance(contract, dict) and contract.get('bank') not in ['LIZ', 'LOM', 'MKO', 'SUG', None] and contract.get('contract_date')
    ]
    
    if valid_loans:
        disb_bank_loan_wo_tbc = sum(
            pd.to_numeric(loan.get('loan_summa', 0), errors='coerce') or 0
            for loan in valid_loans
        )
        disb_bank_loan_wo_tbc = disb_bank_loan_wo_tbc if valid_loans else -3
    
    # Считаем day_sinlastloan
    loans = [
        contract for contract in contracts if isinstance(contract, dict) and contract.get('summa') is not None
    ]
    
    if loans:
        loan_dates = [
            pd.to_datetime(loan['contract_date']).tz_localize(None)
            for loan in loans if pd.notnull(loan['contract_date'])
        ]
        
        if loan_dates:
            last_loan_date = max(loan_dates)
            day_sinlastloan = (application_date - last_loan_date).days if last_loan_date else -1

    return {
        'tot_claim_cnt_l180d': tot_claim_cnt_l180d,
        'disb_bank_loan_wo_tbc': disb_bank_loan_wo_tbc,
        'day_sinlastloan': day_sinlastloan
    }

In [4]:
data_df[['tot_claim_cnt_l180d', 'disb_bank_loan_wo_tbc', 'day_sinlastloan']] = data_df.apply(calculate_features, axis=1, result_type='expand')

In [5]:
data_df = data_df.drop(columns=['application_date', 'contracts'])

In [6]:
data_df.to_csv('contract_features.csv', index=False)

In [7]:
data_df

,id,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
0,2925210.0,-3.0,-1.0,-1.0
1,2925211.0,60.0,0.0,427.0
2,2925212.0,-3.0,-1.0,-1.0
3,2925213.0,60.0,0.0,427.0
4,2925214.0,-3.0,-1.0,-1.0
...,...,...,...,...
995,2926205.0,-3.0,217320883.0,729.0
996,2926206.0,-3.0,-1.0,-1.0
997,2926207.0,-3.0,-1.0,-1.0
998,2926208.0,-3.0,-1.0,-1.0
